In [51]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List, Dict
from collections import defaultdict
import nest_asyncio
nest_asyncio.apply()

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

import time
import tabulate
from qubx import lookup, logger
from qubx.core.basics import Position, ZERO_COSTS
from qubx.core.loggers import PositionsDumper, LogsWriter
from qubx.core.series import time_as_nsec
from qubx.core.utils import time_to_str, time_delta_to_str, recognize_timeframe

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
positions = [
    Position(lookup.find_symbol('BINANCE', s), ZERO_COSTS) for s in ['BTCUSDT', 'ETHUSDT']
] 

positions[0].change_position_by(0, 0.05, 63000)
positions[1].change_position_by(0, 0.5, 3200)

0

In [53]:
class PositionsWriter(LogsWriter):

    def write_data(self, log_type, account_id: str, strategy_id: str, data: List[Dict[str, Any]]):
        table = defaultdict(list)
        total_pnl, total_mkv = 0, 0
        
        for r in data:
            table['Symbol'].append(r['instrument_id'])
            table['Time'].append(r['timestamp'])
            table['Quantity'].append(r['quantity'])
            table['AvgPrice'].append(r['avg_position_price'])
            table['LastPrice'].append(r['current_price'])
            table['PnL'].append(r['pnl_quoted'])
            table['MarketValue'].append(r['market_value_quoted'])
            total_pnl += r['pnl_quoted']
            total_mkv += r['market_value_quoted']

        table['Symbol'].append('TOTAL')
        table['PnL'].append(total_pnl)
        table['MarketValue'].append(total_mkv)

        # - write to database table here
        print(f" ::: Strategy {strategy_id} @ {account_id} :::")
        print(tabulate.tabulate(table, ['Symbol', 'Time', 'Quantity', 'AvgPrice', 'LastPrice', 'PnL', 'MarketValue'], tablefmt='rounded_grid'))

In [54]:
pos_dumper = PositionsDumper('test', 'strategy1', '5Sec', PositionsWriter())
pos_dumper.attach_positions(*positions)

In [55]:
for _ in range(30):
    t = pd.Timestamp('now').asm8
    for p in positions:
        p.update_market_price(t, p.last_update_price + 10, 1)
    pos_dumper.store(t)
    time.sleep(0.5)

 ::: Strategy strategy1 @ test :::
╭──────────┬────────────────────────────┬────────────┬────────────┬─────────────┬───────┬───────────────╮
│ Symbol   │ Time                       │   Quantity │   AvgPrice │   LastPrice │   PnL │   MarketValue │
├──────────┼────────────────────────────┼────────────┼────────────┼─────────────┼───────┼───────────────┤
│ BTCUSDT  │ 2024-04-22T17:58:44.238336 │       0.05 │      63000 │       63010 │   0.5 │        3150.5 │
├──────────┼────────────────────────────┼────────────┼────────────┼─────────────┼───────┼───────────────┤
│ ETHUSDT  │ 2024-04-22T17:58:44.238336 │       0.5  │       3200 │        3210 │   5   │        1605   │
├──────────┼────────────────────────────┼────────────┼────────────┼─────────────┼───────┼───────────────┤
│ TOTAL    │                            │            │            │             │   5.5 │        4755.5 │
╰──────────┴────────────────────────────┴────────────┴────────────┴─────────────┴───────┴───────────────╯
 ::: Strate

In [49]:
pos_dumper.store(pd.Timestamp('now').asm8)

 ::: Strategy strategy1 @ test :::
╭──────────┬────────────────────────────┬────────────┬────────────┬─────────────┬───────┬───────────────╮
│ Symbol   │ Time                       │   Quantity │   AvgPrice │   LastPrice │   PnL │   MarketValue │
├──────────┼────────────────────────────┼────────────┼────────────┼─────────────┼───────┼───────────────┤
│ BTCUSDT  │ 2024-04-22T16:21:05.465011 │       0.05 │      63000 │       63600 │    30 │          3180 │
├──────────┼────────────────────────────┼────────────┼────────────┼─────────────┼───────┼───────────────┤
│ ETHUSDT  │ 2024-04-22T16:21:05.465011 │       0.5  │       3200 │        3800 │   300 │          1900 │
├──────────┼────────────────────────────┼────────────┼────────────┼─────────────┼───────┼───────────────┤
│ TOTAL    │                            │            │            │             │   330 │          5080 │
╰──────────┴────────────────────────────┴────────────┴────────────┴─────────────┴───────┴───────────────╯
